In [1]:
import pandas as pd
from utils.pdf import extract_text_from_url, read_table
import os
import uuid
from pdfminer.high_level import extract_text
import re
df = pd.read_csv('raw_data/ctsv.csv')

In [3]:
df["title"]

0                         Giới thiệu Công tác Sinh viên
1                                 Các hoạt động nổi bật
2                               Logo footer portal.ctsv
3     Giới thiệu mạng lưới hỗ trợ  tìm kiếm việc làm...
4                                  Giới thiệu app iCTSV
                            ...                        
74    Lịch các buổi sinh hoạt công dân bổ sung ...
75    Hướng dẫn dự Lễ tốt nghiệp và hồ sơ tô...
76                Hướng dẫn trả hồ sơ SV ra trường
77         Lịch khám sức khỏe K69 (21.10 - 2.11.24)
78    Hướng dẫn giải đáp các thắc mắc về học t...
Name: title, Length: 79, dtype: object

In [2]:
from tabula import read_pdf
 
#reads table from pdf file
tables = read_pdf("pdf_files/QD_ngoai_ngu_tu_K68_CQ_final.pdf",pages="all") #address of pdf file

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


In [7]:
for table in tables[3:4]:
    # for index, row in table.iterrows():
    #     print(row)
    #     print('-----------------')
    # table = table.drop(table.columns[0], axis=1)
    print(table.to_markdown(index=False))
    break


| Số tín chỉ tích lũy   | Trình độ tiếng Anh yêu cầu                         |
|:----------------------|:---------------------------------------------------|
| Đến 63TC              | + Đạt tất cả học phần Tiếng Anh cơ sở gồm: FL1131, |
| nan                   | FL1132, FL1133                                     |
| nan                   | + Hoặc đạt tối thiểu Bậc 2.1                       |


In [6]:
for row, item in df.iterrows():
    clean_text = ''
    data = item['content']
    parts = re.split(r'Điều \d+\.\s', data)
    for index, part in enumerate(parts):
        part = part.strip()
        if part == '':
            continue
        clean_text += 'Điều ' + str(index) + '.'
        docs = part.split('\n')
        for doc in docs:
            doc = doc.strip()
            if doc == '':
                continue
            if not doc[0].isdigit() and not doc[0].islower() and not doc[0].isupper():
                continue 
            doc = doc.replace('  ', ' ')
            pattern = r'^\d+\.'
            pattern2 = r'^[A-Za-z]\)'
            if re.match(pattern, doc):
                clean_text += '\n' + doc
            elif re.match(pattern2, doc):
                clean_text += '\n' + doc
            else:
                clean_text += ' ' + doc
        clean_text += '\n\n'
    name = item['title']
    if name == 'Quy định ngoại ngữ':
        with open(f'clean_data/{name}.txt', 'w', encoding='utf-8') as f:
            f.write(clean_text)